In [1]:
%run load_dependencies.ipynb
%run misc_functions.ipynb
%run connect_to_api.ipynb

In [2]:
%run tweepy_tweets_getter.ipynb

In [29]:
def analyst_search(mode = 'Hitcache'):
    to_write = {}
    if mode == 'Hitcache':
        obj_type = input('Enter one of the following object types to search:\n1. hashtags\n2. urls\n3. mentions\n4. media \n5. bigrams\n6. trigrams\n7. retweets\n')
        search_term = input('Enter search term: ')
        search_term = search_term.lower()
        days = int(input('Enter how many days before now to check: '))
        print('...Fetching results')
        results = check_hitcache(obj_type,search_term,days,live=True,limit = False)
        results['link'] = [f'https://twitter.com/a/status/{x}' for x in results.message_id]
        to_write['by segment'] = pd.pivot_table(results,index = ['map_id','hit_value','cluster_id'],values = ['message_id','node_id'],aggfunc = pd.Series.nunique)
        to_write['by object'] = pd.pivot_table(results,index = ['hit_value'],values = ['message_id','node_id'],aggfunc = pd.Series.nunique)
        results['date'] = [str(x)[:10] for x in results.hit_time]
        to_write['over time'] = pd.pivot_table(results,index = ['hit_value'],columns = 'date',values = 'node_id',aggfunc = pd.Series.nunique,margins = True).fillna(0)        
        
        to_write['by segment'] = to_write['by segment'].reset_index().sort_values(['map_id','node_id'],ascending = False)
        to_write['by segment'].columns = ['map_id','object_value','cluster_id','tweets','users']
        to_write['by object'] = to_write['by object'].reset_index().sort_values('node_id',ascending = False)
        to_write['by object'].columns = ['hit_value','tweets','users']
        to_write['over time'] = to_write['over time'].reset_index().sort_values('All',ascending = False)
        to_write['raw data'] = results
        write_to_sheet(to_write,f'hitcache_{obj_type}_{search_term}')
    elif mode == 'Twitter':
        search_term = input('Enter search term: ')
        lim = input('If you would like to limit the results, enter a limit, else hit Enter: ')
        if lim == '':
            results = search_tweets(search_term)
        else:
            lim = int(lim)
            results = search_tweets(search_term,limit = lim)
       
        results['link'] = [f'https://twitter.com/a/status/{x}' for x in results.tweet_id]
        results['date'] = [str(x)[:10] for x in results.time]

        temp = pd.pivot_table(results,index = ['retweet_screen_name','retweet_tweet_id'],values = ['tweet_id'],aggfunc = pd.Series.nunique).sort_values('tweet_id',ascending = False)
        rts = pd.pivot_table(results,index = ['retweet_screen_name','retweet_tweet_id'],values = ['retweet_count'],aggfunc = pd.Series.mean).sort_values('retweet_count',ascending = False)
        rts.retweet_count = rts.retweet_count.astype(int)
        temp = temp.merge(rts,left_index = True, right_index = True)
        temp.columns = ['retweets (this week)','retweets (all time)']
        to_write['top accounts'] = temp.reset_index()
        temp = pd.pivot_table(results,index = ['date'],values = ['tweet_id'],aggfunc = pd.Series.nunique).fillna(0)
        temp.columns = ['tweets']
        to_write['over time'] = temp.reset_index()
        to_write['raw data'] = results
        write_to_sheet(to_write,f'twitter_{search_term}')
    return to_write

In [46]:
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

In [76]:
print(f"{bcolors.WARNING}{bcolors.BOLD}Warning: No active bill remain. Continue?{bcolors.ENDC}")


In [77]:
int('250 ')

250